In [1]:
import logging
from sys import stdout

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler(stdout)
console_handler.setFormatter(formatter)

logger = logging.getLogger('opensky.producer')
logger.addHandler(console_handler)
logger.setLevel('DEBUG')



In [2]:
from opensky_api import OpenSkyApi
from kafka import KafkaProducer
import json
from time import sleep
import pyarrow as pa
import os
from datetime import datetime
import shutil
    
# # Topics/Brokers
topic_real_time_states = 'real-time-states'
topic_raw_json = 'raw_json'
topic_flat_json = 'flat_json'
# topic_test = 'topic_test'
brokers = ['localhost:9092']


size_mb = lambda x : x/1024/1024

API = OpenSkyApi('livgeni', '1abc23')

from requests.exceptions import ReadTimeout

def open_sky_generator():
    while True:
        try:
            states = API.get_states()
        except ReadTimeout as rte:
            logger.warning(rte)
        except Exception as e:
            logger.error(e)
        else:
            yield states
        sleep(10)


RED_FIELDS = ['origin_country', 'sensors']
YELLOW_FIELDS = ['time_position', 'heading', 'vertical_rate', 'spi']
IGNORE_FIELDS = RED_FIELDS + YELLOW_FIELDS


def opensky_to_dict(opensky_states):
    """returns a list of dictionaries"""
    svdl = list()
    
    try:
        for sv in opensky_states.states:
            svd = dict(time = opensky_states.time)
            for key in sv.keys:
                if key not in IGNORE_FIELDS:
                    val = sv.__dict__[key]
                    if type(val) == bool:
                        svd[key] = 0 if val == False else 1
                    else:
                        svd[key] = val if type(val) != str else val.strip()
            svdl += [svd]
    except Exception as e:
        log.error(e)
    return svdl
#     state_vector_json_list = json.dumps(svdl)
#     return state_vector_json_list


        
class OpenskyArchiver:
    
    def __init__(self):
        # hdfs config
        self.hdfs_config = dict(
                            hdfs_host='localhost',
                            hdfs_port=8020,
                            hdfs_user='hdfs',
                            hdfs_driver='libhdfs',
                            hdfs_archive_path = '/FinalProject/Archive'
                               )
        # local file configs
        self.local_files_folder = os.path.abspath('/home/naya/tutorial/open-sky/tmp_storage')
        self.local_files_prefix = 'opensky_state_vectors_json'
        self.local_file_size_threshold = 100 #MB
        self.gen_time_suffix = lambda : datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
        self.gen_local_file_path = lambda : os.path.join\
                                    (self.local_files_folder,\
                                     f'{self.local_files_prefix}_{self.gen_time_suffix()}.json')
        self.local_file_path = self.gen_local_file_path()
        
        # init local and hdfs folders
        fs = pa.hdfs.connect(
            host=self.hdfs_config['hdfs_host'],
            port=self.hdfs_config['hdfs_port'], 
            user=self.hdfs_config['hdfs_user'], 
            kerb_ticket=None, 
            driver=self.hdfs_config['hdfs_driver'], 
            extra_conf=None)

        # Create local folder if not exists
        if os.path.exists(self.local_files_folder):
            shutil.rmtree(self.local_files_folder)
        os.makedirs(self.local_files_folder)

        # create hdfs folder if not exist
        hdfs_archive_path = self.hdfs_config['hdfs_archive_path']
        if not fs.exists(hdfs_archive_path):
            fs.mkdir(hdfs_archive_path, create_parents=True)
        
        self.fs = fs
        
    def archive_data(self, jsn_str):
        """
        load as json current file if exists, and append the new json string and dump back to the file
        if file does not exist create new and dump the new file    
        """ 
        if os.path.isfile(self.local_file_path) and os.path.getsize(self.local_file_path) > 0:
            with open(self.local_file_path, 'r') as local_file:
                file_jsn = json.load(local_file)
        else:
            file_jsn = json.loads('[]')
        with open(self.local_file_path, 'w') as local_file:
            new_jsn = json.loads(jsn_str)
            file_jsn.extend(new_jsn)
            json.dump(file_jsn, local_file)
#             local_file.write(jsn_str)
        # test if target file size reached
        if size_mb(os.path.getsize(self.local_file_path)) >= self.local_file_size_threshold:
            # upload to hdfs
            logger.debug(f'Uploading to HDFS')
            with open(self.local_file_path, 'rb') as source:
                target_file_name = os.path.split(self.local_file_path)[1]
                target_full_path = f'{self.hdfs_config["hdfs_archive_path"]}/{target_file_name}'
                self.fs.upload(target_full_path, source)
            logger.debug(f'Uploaded to : {target_full_path}')
            # delete the local file
            logger.debug(f'deleting local file {self.local_file_path}')
            os.remove(self.local_file_path)
            # new name for new file
            self.local_file_path = self.gen_local_file_path()
            logger.debug(f'generated new local_file_name :{os.path.split(self.local_file_path)[1]}')

## Save to Archiving Folder in HDFS

1. Append to a local file until reaching 100MB
2. When reached - upload to HDFS using pyarrow and open a new file

In [ ]:
producer = KafkaProducer(bootstrap_servers=brokers, max_request_size = 4299162)

archiver = OpenskyArchiver()

for states in open_sky_generator():
    if states is None:
        logger.warning('Received empty states vector')
        continue
    
    message_json = json.dumps(opensky_to_dict(states))
    # First archive the message
    archiver.archive_data(message_json)
    # Then send to kafka
    send_result = producer.send(topic_real_time_states, value = message_json.encode('utf-8'))
    if send_result.exception:
        logger.error(f'producer send ecxeption: {send_result.exception}')
    logger.debug(f"sent time : {datetime.fromtimestamp(states.time)} ; len of sent message : {len(message_json)}")
    

2019-12-29 17:19:03,817 - opensky.producer - ERROR - HTTPSConnectionPool(host='opensky-network.org', port=443): Max retries exceeded with url: /api/states/all?time=0 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
2019-12-29 17:19:14,195 - opensky.producer - ERROR - HTTPSConnectionPool(host='opensky-network.org', port=443): Max retries exceeded with url: /api/states/all?time=0 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
2019-12-29 17:19:35,648 - opensky.producer - DEBUG - sent time : 2019-12-29 17:19:30 ; len of sent message : 2089352
2019-12-29 17:19:53,734 - opensky.producer - DEBUG - sent time : 2019-12-29 17:19:45 ; len of sent message : 2088778
2019-12-29 17:20:07,322 - opensky.producer - DEBUG - sent time : 2019-12-29 17:20:00 ; len of sent message : 2087234
2019-12-29 17:20:21,195 - opensky.producer - DEBUG - sent time : 2019-12-29 17:20:15 ; len of sent message : 2088637
2019-12-29 17:20:49,625 - op

2019-12-29 17:36:43,034 - opensky.producer - DEBUG - sent time : 2019-12-29 17:36:30 ; len of sent message : 2106241
2019-12-29 17:37:08,447 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 17:37:19,027 - opensky.producer - ERROR - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
2019-12-29 17:37:29,486 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:37:39,955 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:37:50,452 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:38:08,880 - opensky.producer - DEBUG - sent time : 2019-12-29 17:38:00 ; len of sent message : 2121964
2019-12-29 17:38:34,283 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 17:39:00,802 - opensky.producer - DEBUG - sent time : 20

2019-12-29 17:54:31,808 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:54:52,928 - opensky.producer - DEBUG - sent time : 2019-12-29 17:54:40 ; len of sent message : 2198564
2019-12-29 17:55:03,410 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:55:13,879 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:55:24,350 - opensky.producer - WARNING - Received empty states vector
2019-12-29 17:55:46,742 - opensky.producer - DEBUG - sent time : 2019-12-29 17:55:30 ; len of sent message : 2208055
2019-12-29 17:56:17,849 - opensky.producer - DEBUG - sent time : 2019-12-29 17:56:05 ; len of sent message : 2209595
2019-12-29 17:56:28,253 - opensky.producer - ERROR - ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
2019-12-29 17:56:53,623 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 17:57

2019-12-29 18:13:29,905 - opensky.producer - WARNING - Received empty states vector
2019-12-29 18:13:40,372 - opensky.producer - WARNING - Received empty states vector
2019-12-29 18:13:54,256 - opensky.producer - DEBUG - sent time : 2019-12-29 18:13:45 ; len of sent message : 2381478
2019-12-29 18:14:23,246 - opensky.producer - DEBUG - sent time : 2019-12-29 18:14:15 ; len of sent message : 2378652
2019-12-29 18:14:48,742 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 18:15:14,122 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 18:15:39,512 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 18:15:55,927 - opensky.producer - DEBUG - sent time : 2019-12-29 18:15:45 ; len of sent message : 2379125
2019-12-29 18:16:06,460 - opens

2019-12-29 18:34:25,712 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 18:34:47,507 - opensky.producer - DEBUG - sent time : 2019-12-29 18:34:35 ; len of sent message : 2756150
2019-12-29 18:34:57,775 - opensky.producer - ERROR - HTTPSConnectionPool(host='opensky-network.org', port=443): Max retries exceeded with url: /api/states/all?time=0 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))
2019-12-29 18:35:08,265 - opensky.producer - WARNING - Received empty states vector
2019-12-29 18:35:18,800 - opensky.producer - WARNING - Received empty states vector
2019-12-29 18:35:29,272 - opensky.producer - WARNING - Received empty states vector
2019-12-29 18:35:54,629 - opensky.producer - WARNING - HTTPSConnectionPool(host='opensky-network.org', port=443): Read timed out. (read timeout=15.0)
2019-12-29 18:36:04,875 - opensky.producer - ERROR - HTTPSConnectionPool(host='

...

2019-12-30 21:08:47,640 - opensky.producer - DEBUG - sent time : 2019-12-30 21:08:40 ; len of sent message : 2207239
2019-12-30 21:09:02,126 - opensky.producer - DEBUG - sent time : 2019-12-30 21:08:55 ; len of sent message : 2208530
2019-12-30 21:09:17,492 - opensky.producer - DEBUG - sent time : 2019-12-30 21:09:10 ; len of sent message : 2211520
2019-12-30 21:09:36,089 - opensky.producer - DEBUG - sent time : 2019-12-30 21:09:25 ; len of sent message : 2210508
2019-12-30 21:09:51,909 - opensky.producer - DEBUG - sent time : 2019-12-30 21:09:45 ; len of sent message : 2208580
2019-12-30 21:10:09,085 - opensky.producer - DEBUG - sent time : 2019-12-30 21:10:00 ; len of sent message : 2202994
2019-12-30 21:10:25,880 - opensky.producer - DEBUG - sent time : 2019-12-30 21:10:15 ; len of sent message : 2203354
2019-12-30 21:10:42,146 - opensky.producer - DEBUG - sent time : 2019-12-30 21:10:35 ; len of sent message : 2202013
2019-12-30 21:11:00,744 - opensky.producer - DEBUG - sent time :

2019-12-30 21:29:43,912 - opensky.producer - DEBUG - sent time : 2019-12-30 21:29:30 ; len of sent message : 2015652
2019-12-30 21:30:01,762 - opensky.producer - DEBUG - sent time : 2019-12-30 21:29:50 ; len of sent message : 2020179
2019-12-30 21:30:20,958 - opensky.producer - DEBUG - sent time : 2019-12-30 21:30:10 ; len of sent message : 2018141
2019-12-30 21:30:39,383 - opensky.producer - DEBUG - sent time : 2019-12-30 21:30:30 ; len of sent message : 2021891
2019-12-30 21:30:58,872 - opensky.producer - DEBUG - sent time : 2019-12-30 21:30:45 ; len of sent message : 2025244
2019-12-30 21:31:18,769 - opensky.producer - DEBUG - sent time : 2019-12-30 21:31:05 ; len of sent message : 2025201
2019-12-30 21:31:38,537 - opensky.producer - DEBUG - sent time : 2019-12-30 21:31:24 ; len of sent message : 2024764
2019-12-30 21:31:59,031 - opensky.producer - DEBUG - sent time : 2019-12-30 21:31:45 ; len of sent message : 2025202
2019-12-30 21:32:18,558 - opensky.producer - DEBUG - sent time :

2019-12-30 21:51:05,684 - opensky.producer - DEBUG - sent time : 2019-12-30 21:50:54 ; len of sent message : 2031940
2019-12-30 21:51:26,701 - opensky.producer - DEBUG - sent time : 2019-12-30 21:51:15 ; len of sent message : 2031644
2019-12-30 21:51:49,356 - opensky.producer - DEBUG - sent time : 2019-12-30 21:51:35 ; len of sent message : 2028906
2019-12-30 21:52:11,955 - opensky.producer - DEBUG - sent time : 2019-12-30 21:51:55 ; len of sent message : 2034697
2019-12-30 21:52:34,257 - opensky.producer - DEBUG - sent time : 2019-12-30 21:52:20 ; len of sent message : 2036766
2019-12-30 21:52:57,509 - opensky.producer - DEBUG - sent time : 2019-12-30 21:52:40 ; len of sent message : 2039170
2019-12-30 21:53:19,299 - opensky.producer - DEBUG - sent time : 2019-12-30 21:53:05 ; len of sent message : 2045331
2019-12-30 21:53:41,706 - opensky.producer - DEBUG - sent time : 2019-12-30 21:53:24 ; len of sent message : 2043070
2019-12-30 21:54:05,123 - opensky.producer - DEBUG - sent time :

2019-12-30 22:13:04,404 - opensky.producer - DEBUG - sent time : 2019-12-30 22:12:49 ; len of sent message : 2041278
2019-12-30 22:13:31,375 - opensky.producer - DEBUG - sent time : 2019-12-30 22:13:10 ; len of sent message : 2040247
2019-12-30 22:13:56,702 - opensky.producer - DEBUG - sent time : 2019-12-30 22:13:40 ; len of sent message : 2047449
2019-12-30 22:14:21,732 - opensky.producer - DEBUG - Uploading to HDFS
2019-12-30 22:14:22,122 - opensky.producer - DEBUG - Uploaded to : /FinalProject/Archive/opensky_state_vectors_json_2019-12-30_21_57_20.json
2019-12-30 22:14:22,124 - opensky.producer - DEBUG - deleting local file /home/naya/tutorial/open-sky/tmp_storage/opensky_state_vectors_json_2019-12-30_21_57_20.json
2019-12-30 22:14:22,147 - opensky.producer - DEBUG - generated new local_file_name :opensky_state_vectors_json_2019-12-30_22_14_22.json
2019-12-30 22:14:22,280 - opensky.producer - DEBUG - sent time : 2019-12-30 22:14:04 ; len of sent message : 2048076
2019-12-30 22:14:3

2019-12-30 22:32:05,354 - opensky.producer - DEBUG - sent time : 2019-12-30 22:32:00 ; len of sent message : 2085537
2019-12-30 22:32:19,950 - opensky.producer - DEBUG - sent time : 2019-12-30 22:32:15 ; len of sent message : 2086668
2019-12-30 22:32:35,251 - opensky.producer - DEBUG - sent time : 2019-12-30 22:32:25 ; len of sent message : 2089042
2019-12-30 22:32:48,942 - opensky.producer - DEBUG - sent time : 2019-12-30 22:32:40 ; len of sent message : 2087943
2019-12-30 22:33:03,190 - opensky.producer - DEBUG - sent time : 2019-12-30 22:32:55 ; len of sent message : 2089873
2019-12-30 22:33:18,034 - opensky.producer - DEBUG - sent time : 2019-12-30 22:33:10 ; len of sent message : 2083176
2019-12-30 22:33:34,136 - opensky.producer - DEBUG - sent time : 2019-12-30 22:33:25 ; len of sent message : 2082619
2019-12-30 22:33:50,631 - opensky.producer - DEBUG - sent time : 2019-12-30 22:33:40 ; len of sent message : 2088024
2019-12-30 22:34:06,200 - opensky.producer - DEBUG - sent time :